### 来自廖雪峰官方网站
https://www.liaoxuefeng.com/wiki/1016959663602400/1017630786314240

在多线程环境下，每个线程都有自己的数据。一个线程使用自己的局部变量比使用全局变量好，因为局部变量只有线程自己能看见，不会影响其他线程，而全局变量的修改必须加锁。  
但是局部变量也有问题，就是在函数调用的时候，参数传递起来很麻烦。
如果用一个全局`dict`存放所有的局部对象，然后以`thread`自身作为`key`获得线程对应的局部对象如何？ -> 太丑了  
`ThreadLocal`应运而生，不用查找`dict`，`ThreadLocal`帮你自动做这件事：

In [3]:
import threading
local = threading.local()


def process_student():
    student = local.student
    print("Hello, {student} (in {thread})".format(student=student, thread=threading.current_thread().name))

def process_thread(name: str):
    local.student = name
    process_student()

t1 = threading.Thread(target=process_thread, args=("Bob", ))
t2 = threading.Thread(target=process_thread, args=("Alice", ))
t1.start()
t2.start()
t1.join()
t2.join()

Hello, Bob (in Thread-8)
Hello, Alice (in Thread-9)


全局变量`local`就是一个`ThreadLocal`对象，每个Thread对它都可以读写`student`属性，但互不影响。可以把`local`看成全局变量，但每个属性如`local.student`都是线程的局部变量，可以任意读写而互不干扰，也不用管理锁的问题，`ThreadLocal`内部会处理。  
可以理解为全局变量`local`是一个`dict`，不但可以用`local.student`，还可以绑定其他变量，如`local.teacher`等等。  

*`ThreadLocal`最常用的地方就是为每个线程绑定一个数据库连接，HTTP请求，用户身份信息等，这样一个线程的所有调用到的处理函数都可以非常方便地访问这些资源。*